In [3]:
import numpy as np
from scipy.optimize import curve_fit
from numpy.polynomial import Polynomial

r2_rez = dict()
coeffs_rez = dict()

In [4]:
coeffs = [-0.5, 3]
domain = (-10, 10)
N = 20

In [5]:
p = Polynomial(coeffs[::-1], domain=domain, window=domain)
p

Polynomial([ 3. , -0.5], domain=[-10.,  10.], window=[-10.,  10.])

In [6]:
def exp_func(p: Polynomial, c: float, N: int = N) -> np.ndarray:
    return np.exp(p.linspace(N)[1]) + c

In [7]:
xdata = dict()
ydata = dict()

xdata['deg 1'], ydata['poly'] = p.linspace(N)
xdata['deg 1'] = xdata['deg 1'].reshape(-1, 1)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
import pandas as pd

noize = np.random.normal(size=N, scale=np.mean(abs(ydata['poly']))/4)
ydata_noized = ydata['poly'] + noize

In [9]:
import plotly.graph_objects as go

plt0 = go.Figure()
plt0.add_trace(
    go.Scatter(
        x=xdata['deg 1'][:, 0],
        y=ydata_noized,
        mode='markers',
        name='Data',
        marker=dict(
            color='LightGrey',
            size=10
        ),
    )
)
# plt.update_yaxes(type="log")
plt0.show()

In [10]:
degs = dict()
reg = dict()
r2 = dict()
plt = dict()
new_y = dict()
new_x = dict()

for j in range(1, 6):
    i, k = f'deg {j}', 'poly'
    if j == 5:
        i, k = 'exp', 'exp'
        xy = pd.DataFrame({'x': xdata['deg 1'][:, 0], 'y': ydata_noized})
        xy = xy[xy['y'] > 0]
        trans = FunctionTransformer(np.log, validate=True)
        ydata['exp'] = trans.fit_transform(np.array(xy['y']).reshape(-1, 1))
        xdata['exp'] = np.array(xy['x']).reshape(-1, 1)
        new_x['exp'] = x
        reg['exp'] = LinearRegression().fit(xdata['exp'], ydata['exp'])
        new_y['exp'] = np.exp(reg['exp'].predict(new_x['exp']))[:, 0]
        r2['exp'] = r2_score(np.array(xy['y']), np.exp(reg['exp'].predict(xdata[i])))
    else:
        if j == 1:
            x = np.linspace(*domain, N * 20).reshape(-1, 1)
            new_x['deg 1'] = x
        else:
            degs[i] = PolynomialFeatures(degree=j)
            new_x[i] = degs[i].fit_transform(new_x['deg 1'])
            xdata[i] = degs[i].fit_transform(xdata['deg 1'])
        
        reg[i] = LinearRegression().fit(xdata[i], ydata[k])
        r2[i] = r2_score(ydata_noized, reg[i].predict(xdata[i]))
        new_y[i] = reg[i].predict(new_x[i])
        
    r2_now = r2[i]

    plt[i] = go.Figure()
    plt[i].add_trace(
        go.Scatter(
            x=xdata['deg 1'][:, 0],
            y=ydata_noized,
            mode='markers',
            name='Data',
            marker=dict(
                color='LightGrey',
                size=10
            ),
        )
    )

    trace = go.Scatter(
        x=new_x['deg 1'][:, 0],
        y=new_y[i],
        mode='lines',
        name=f'Prediction degree {j}, R2 = {r2_now:.3f}' if j != 5 else f'Prediction exp, R2 = {r2_now:.3f}'
    )

    plt[i].add_trace(trace)
    if j == 5:
        plt[i].write_image(f'images/{len(coeffs) - 1}_degree_poly_exp_fit.png', width=1024, height=512, scale=4)
    else:
        plt[i].write_image(f'images/{len(coeffs) - 1}_degree_poly_{j}_degree_fit.png', width=1024, height=512, scale=4)
    plt0.add_trace(trace)

In [11]:
plt0.write_image(f'images/{len(coeffs) - 1}_degree_poly_all_together.png', width=1024, height=512, scale=4)
plt0.show()

In [470]:
r2_rez[len(coeffs) - 1] = [f'{key} - {value}' for key, value in r2.items()]
coeffs_rez[len(coeffs) - 1] = coeffs

In [471]:
from pprint import pprint

pprint(r2_rez)
pprint(coeffs_rez)

{1: ['deg 1 - 0.9426711700857244',
     'deg 2 - 0.9426711700857245',
     'deg 3 - 0.9426711700857244',
     'deg 4 - 0.9426711700857136',
     'exp - 0.08502006967534181'],
 2: ['deg 1 - 0.9010200056321328',
     'deg 2 - 0.9682248517518471',
     'deg 3 - 0.9682248517518471',
     'deg 4 - 0.968224851751846',
     'exp - 0.6693294079327454'],
 3: ['deg 1 - 0.10466366650607517',
     'deg 2 - 0.12791603079736102',
     'deg 3 - 0.961163381723431',
     'deg 4 - 0.9611633817234209',
     'exp - -0.08746212826076638'],
 4: ['deg 1 - 0.5784239365423052',
     'deg 2 - 0.5786838634695093',
     'deg 3 - 0.8578833661112893',
     'deg 4 - 0.9648199060313363',
     'exp - 0.5178778886773503'],
 'exp': ['deg 1 - 0.1286213775005266',
         'deg 2 - 0.30709326675836357',
         'deg 3 - 0.4969797279171785',
         'deg 4 - 0.6664289716548277',
         'exp - 0.8239630792765567']}
{1: [-0.5, 3],
 2: [0.05, -1, 0],
 3: [0.05, -0.06, -4, 0],
 4: [0.00625, 0.05, -0.6, -1, 0],
 'exp': [0.5